# Capstone Project: Slogan Classifier and Generator

In this capstone project you will train a Long Short-Term Memory (LSTM) model to generate slogans for businesses based on their industry, and also train a classifier to predict the industry based on a given slogan.

##Libraries
We recommend running this notebook using [Google Colab](https://colab.google/) however if you choose to use your local machine you will need to install spaCy before starting.

To install spaCy, refer to the installation instructions provided on the spaCy [website](https://spacy.io/usage). Note you may need to install an older version of Python that is compatible with spaCy. You can create a virtual environment for this project to install the specific version of Python that you need.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam
import spacy # available on Google Colab
from sklearn.model_selection import train_test_split

2025-06-20 16:52:11.024615: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Loading and viewing the dataset

- Load the slogan dataset into a variable called data.
- Extract relevant columns in a variable called df.
- Handle missing values.

Do **not** change the column names.

If you are using Google Colab you will need mount your Google Drive as follows:  
`from google.colab import drive`  
`drive.mount('/content/drive')`  

The path you use when loading your data will look something like this if you are using your Google Drive:  
"/content/drive/MyDrive/Colab Notebooks/slogan-valid.csv"

In [2]:
# Load data from your local machine
data = pd.read_csv("slogan-valid.csv")
data.head()


# Extract relevant columns
data.columns
df = data[['industry', 'output']]

# Drop rows with missing values in either column
df = df.dropna(subset=['industry', 'output'])

# View the cleaned dataset
df.head()


,industry,output
0,computer hardware,Taking Care of Small Business Technology
1,"health, wellness and fitness",Build World-Class Recreation Programs
2,internet,Most Powerful Lead Generation Software for Mar...
3,internet,Hire quality freelancers for your job
4,financial services,"Financial Advisers Norwich, Norfolk"


## Data Preprocessing

Since we are working with textual data, we need software that understands natural language. For this, we'll use a library for processing text called **spaCy**. Using spaCy, we'll break the text into smaller units called tokens that are easier for the machine to process. This process is called **tokenisation**. We'll also convert all text to lowercase and remove punctuation because this information is not necessary for our models. Run the code below, and your dataframe (df) will gain a new column called **'processed_slogan'** which contains the preprocessed text.




In [3]:
# import spacy.cli
# spacy.cli.download("en_core_web_sm")

# Load spaCy model for text processing
nlp = spacy.load("en_core_web_sm")

# Define text preprocessing function
def preprocess_text(text):
    text_lower = text.lower()
    doc = nlp(text_lower)

    processed_tokens = []

    for token in doc:
        if not token.is_punct:
            processed_tokens.append(token.text)

    return " ".join(processed_tokens)

df["processed_slogan"] = df["output"].apply(preprocess_text)

df.head()

,industry,output,processed_slogan
0,computer hardware,Taking Care of Small Business Technology,taking care of small business technology
1,"health, wellness and fitness",Build World-Class Recreation Programs,build world class recreation programs
2,internet,Most Powerful Lead Generation Software for Mar...,most powerful lead generation software for mar...
3,internet,Hire quality freelancers for your job,hire quality freelancers for your job
4,financial services,"Financial Advisers Norwich, Norfolk",financial advisers norwich norfolk


We want our model to generate **industry-specific** slogans. If we use the 'processed_slogan' column as it is, we'll be leaving out crucial context - the industries of the companies behind those slogans. To fix this, we'll create a new **'modified_slogan'** column that adds the industry name to the front of processed slogan.  

For example:  

> industry = 'computer hardware'  
processed_slogan = 'taking care of small business technology'  
modified_slogan = 'computer hardware taking care of small business technology'

Write code in the cell below to achieve this.

In [4]:
# Create a new column by prepending the industry to the processed slogan
df['modified_slogan'] = df['industry'] + ' ' + df['processed_slogan']

# View a few rows to confirm
df[['industry', 'processed_slogan', 'modified_slogan']].head()


,industry,processed_slogan,modified_slogan
0,computer hardware,taking care of small business technology,computer hardware taking care of small busines...
1,"health, wellness and fitness",build world class recreation programs,"health, wellness and fitness build world class..."
2,internet,most powerful lead generation software for mar...,internet most powerful lead generation softwar...
3,internet,hire quality freelancers for your job,internet hire quality freelancers for your job
4,financial services,financial advisers norwich norfolk,financial services financial advisers norwich ...


Now we need to get data to train our model. We have textual data which we will need to represent numerically for our model to learn from it.  
The code below does the following:
1. Tokenizes a dataset of slogans.
2. Converts words to numerical indices.
3. Creates input sequences using the numerical indices.  

Here's how it works. From the 'modified_slogan' column, we take the slogan "computer hardware taking care of small business technology". The tokenisation process will convert words into their corresponding indices:  

<center>

| Word         | Token Index |
|-------------|-------|
| "computer"  | 1     |
| "hardware"  | 2     |
| "taking"    | 3     |
| "care"      | 4     |
| "of"        | 5     |
| "small"     | 6     |
| "business"  | 7     |
| "technology"| 8     |

</center>

So the tokenized list is:

<center>
[1, 2, 3, 4, 5, 6, 7, 8]
</center>

When creating input sequences for training, the loop generates progressively longer sequences.

<center>

| Token Index Sequence               | Corresponding Slogan                                 |
|------------------------------|-----------------------------------------------------|
| [1, 2]                       | "computer hardware"                                |
| [1, 2, 3]                    | "computer hardware taking"                        |
| [1, 2, 3, 4]                 | "computer hardware taking care"                   |
| [1, 2, 3, 4, 5]              | "computer hardware taking care of"                |
| [1, 2, 3, 4, 5, 6]           | "computer hardware taking care of small"          |
| [1, 2, 3, 4, 5, 6, 7]        | "computer hardware taking care of small business" |
| [1, 2, 3, 4, 5, 6, 7, 8]     | "computer hardware taking care of small business technology" |

</center>

Instead of training the model on only **complete slogans**, we provide partial phrases which will help the model learn how words connect over time. This will make it better at predicting the next word when generating slogans.  

Run the cell block below to generate the input sequences. Be sure to read the comments to understand what the code is doing.


In [5]:
'''** Clean up comments'''

# Tokenizer to convert words into numerical values tokens
tokenizer = Tokenizer()

# Tokenizer learns words in dataset
tokenizer.fit_on_texts(df["modified_slogan"])

# Total number of unique words in learned vocabulary
total_words = len(tokenizer.word_index) + 1

# Dictionary mapping words to its numerical index: index based on frequency i.e., more freq => lower index
tokenizer.word_index

# Creating input sequences
# Initialise list to store the input sequences
input_sequences = []

# Iterate over processed slogans
for line in df["modified_slogan"]:

    # Convert slogans to token sequences
    token_list = tokenizer.texts_to_sequences([line])[0] # returns list containing list of words indices; extracting inner list [0]

    # token_list is a list of tokenized word INDICES
    # Building list of progressively longer input sequences for better training
    for i in range(1, len(token_list)):
        input_sequences.append(token_list[:i+1])

The input sequences created above are of **varying lengths**, which will be a problem when training our LSTM model. LSTMs require input sequences of **equal length**. So, we need to **pad** shorter sequences by **prepending zeros** until they match the length of the longest sequence.  

For example, if the longest sequence has **10 tokens**, our padded sequences will look like this:

<center>

| Input Sequence                     | Padded Sequence                         |
|-------------------------------------|-----------------------------------------|
| [1, 2]                              | [0, 0, 0, 0, 0, 0, 0, 0, 1, 2]         |
| [1, 2, 3]                           | [0, 0, 0, 0, 0, 0, 0, 1, 2, 3]         |
| [1, 2, 3, 4]                        | [0, 0, 0, 0, 0, 0, 1, 2, 3, 4]         |
| [1, 2, 3, 4, 5]                     | [0, 0, 0, 0, 0, 1, 2, 3, 4, 5]         |
| [1, 2, 3, 4, 5, 6]                  | [0, 0, 0, 0, 1, 2, 3, 4, 5, 6]         |
| [1, 2, 3, 4, 5, 6, 7]               | [0, 0, 0, 1, 2, 3, 4, 5, 6, 7]         |
| [1, 2, 3, 4, 5, 6, 7, 8]            | [0, 0, 1, 2, 3, 4, 5, 6, 7, 8]         |

</center>

In the cell below, write code that **finds the length of the longest sequence** in **input_sequences** and stores this value in a variable named **max_seq_len**.


In [6]:
# Find the maximum sequence length
max_seq_len = max(len(seq) for seq in input_sequences)

print("Maximum sequence length:", max_seq_len)


Maximum sequence length: 15


Run the cell below to pad the input sequences so they are all the same length as **max_seq_length**.

In [7]:
input_sequences = pad_sequences(input_sequences, maxlen=max_seq_len, padding="pre")

## Training Data for Slogan Generator

The input sequences generated will be used as our training data. Our LSTM needs to learn how to predict the **next word** in a sequence.  

The inputs for our model will be the input sequences **excluding the last token index** and the outputs will be the **last token index**.  

As an example, let us use the input sequence [0, 0, 1, 2, 3, 4, 5, 6, 7, 8] and say it corresponds to the slogan "computer hardware taking care of small business technology". When training the model:

> Our input **x** will be the input sequence [0, 0, 1, 2, 3, 4, 5, 6, 7] corresponding to "computer hardware taking care of small".  
> Our output **y** will be [8] which corresponds to "business".  

In the code cell below, use `input_sequences` to create the following two variables:
1. **X_gen** which contains the input sequences excluding the last token index.
2. **y_gen** which contains the last token index of the input sequence.

In [8]:
# Inputs: all tokens except the last one
X_gen = input_sequences[:, :-1]

# Outputs: the last token of each sequence
y_gen = input_sequences[:, -1]

print("X_gen shape:", X_gen.shape)
print("y_gen shape:", y_gen.shape)

X_gen shape: (34736, 14)
y_gen shape: (34736,)


The model will output the next word of a sequence over a probability distribution. We need to encode our output variable for this to be possible.

In the code cell below, write code that will apply one-hot encoding to **y_gen** using `tf.keras.utils.to_categorical()`. **Maintain the same variable name**.  

*Hint: set the `num_classes` (number of classes) parameter to the total number of unique words in the learned vocabulary. You can access this value through a variable that was created when generating input sequences earlier.*

In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

# One-hot encode the output sequences
y_gen = to_categorical(y_gen, num_classes=total_words)

print("One-hot encoded y_gen shape:", y_gen.shape)

One-hot encoded y_gen shape: (34736, 6046)


## Slogan Generator Architecture

In the code cell that follows, configure the LSTM following these steps:

1. Create a sequential model using `tf.keras.models.Sequential()`. This model will have an embedding layer, two LSTM layers, and a dense output layer.
2. Add an embedding layer that converts words into dense vector representations. This layer should:
> *   Have `total_words`as the vocabulary size.
> *   Use 100 as an embedding dimension.
> *   Takes an input length of `max_seq_len - 1` (excludes the target word).
3. Add two LSTM layers.
> *   The first LSTM layer should have 150 **and** set `return_sequences` to `True`.
> *   The second LSTM layer should have 100 units.
4. Add a dense output layer which:
> *   Uses `total_words` as the number of units (one for each word in the vocabulary).
> *   Uses a softmax activation function.
5. Use `Sequential` to put everything together in the correct order to complete the architecture of the LSTM model called **gen_model**.


In [10]:
gen_model = Sequential()

# Embedding layer: vocab size = total_words, embedding dim = 100, input length = max_seq_len - 1
gen_model.add(Embedding(input_dim=total_words, output_dim=100, input_length=max_seq_len - 1))

# First LSTM layer with 150 units, returning sequences
gen_model.add(LSTM(150, return_sequences=True))

# Second LSTM layer with 100 units
gen_model.add(LSTM(100))

# Dense output layer with softmax activation for predicting next word
gen_model.add(Dense(total_words, activation='softmax'))


In the code cell below, compile `gen_model` using `categorical_crossentropy` loss, an Adam optimiser, and an appropriate metric of your choice.


In [11]:
# Set the loss to categorical crossentropy, uses the 
# Adam optimizer with default parameters, 
# and tracks accuracy during training.

from tensorflow.keras.optimizers import Adam

gen_model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

## Slogan Generation

In the code cell below, fit the compiled model on the inputs and outputs, setting the **number of epochs to 50**.

In [12]:
history = gen_model.fit(
    X_gen,
    y_gen,
    epochs=50,
    batch_size=64,
    validation_split=0.2
)

Epoch 1/50
435/435 [==============================] - 65s 131ms/step - loss: 7.2783 - accuracy: 0.0629 - val_loss: 6.9807 - val_accuracy: 0.0645
Epoch 2/50
435/435 [==============================] - 50s 115ms/step - loss: 6.6953 - accuracy: 0.0740 - val_loss: 6.8938 - val_accuracy: 0.0924
Epoch 3/50
435/435 [==============================] - 52s 119ms/step - loss: 6.4160 - accuracy: 0.0920 - val_loss: 6.7927 - val_accuracy: 0.0937
Epoch 4/50
435/435 [==============================] - 49s 112ms/step - loss: 6.2071 - accuracy: 0.1088 - val_loss: 6.7669 - val_accuracy: 0.1187
Epoch 5/50
435/435 [==============================] - 47s 109ms/step - loss: 6.0312 - accuracy: 0.1298 - val_loss: 6.7077 - val_accuracy: 0.1421
Epoch 6/50
435/435 [==============================] - 49s 112ms/step - loss: 5.8610 - accuracy: 0.1500 - val_loss: 6.6501 - val_accuracy: 0.1544
Epoch 7/50
435/435 [==============================] - 50s 114ms/step - loss: 5.6986 - accuracy: 0.1693 - val_loss: 6.6060 - val_ac

We will now define a function called `generate_slogan` which will generate a slogan by predicting one word at a time based on a given starting phrase (the `seed_text`). This function will do this using our trained model, `gen_model`.

Here is a breakdown of how the algorithm works:  

Let us assume the dictionary mapping words to unique indices, `tokenizer.word_index`, looks like this:

> `{'computer': 1, 'hardware': 2, 'taking': 3, 'care': 4, 'of': 5}`

If the model's predicted index for the next word is 3 (`predicted_index = 3`), the loop will:

> Check 'computer' (index 1) → No match  
> Check 'hardware' (index 2) → No match  
> Check 'taking' (index 3) → Match found!  
> Assign output_word = "taking" and exit the loop.  

The `output_word` will be appended to the `seed_text`, and the process will continue to add words to the `seed_text` until we have reached the maximum number of words **or** an invalid prediction occurs.  

Carefully follow the code below and complete the missing parts as guided by the comments.

In [13]:
def generate_slogan(seed_text, max_words=20):
    for _ in range(max_words):

        # Tokenising and padding seed_text
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding="pre")

        # Use your trained model (gen_model) on token_list to predict the probability distribution of the next word
        predictions = gen_model.predict(token_list, verbose=0)

        # Identify the word index with the highest probability
        predicted_index = np.argmax(predictions, axis=-1)[0]

        output_word = None

        # Find the word corresponding to the predicted index
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break

        # Stop if no valid word is found
        if output_word is None:
            break

        # Append predicted word to the seed_text
        seed_text += " " + output_word

    return seed_text

## Training Data for Slogan Classifier

We will now prepare the data we will use to train our classifier. For our classifier, the inputs will come from the `processed_slogans` column of our DataFrame, `df`. The outputs will be the different industry categories under the `industry` column.

In the code cell below, extract the unique values from the `industry` column in the DataFrame and store these in a variable called **industries**.

In [14]:
industries = df['industry'].unique()
print(industries)

['computer hardware' 'health, wellness and fitness' 'internet'
 'financial services' 'mechanical or industrial engineering'
 'marketing and advertising' 'hospital & health care' 'research'
 'information technology and services' 'computer software' 'oil & energy'
 'dairy' 'transportation/trucking/railroad' 'design' 'furniture'
 'professional training & coaching' 'hospitality' 'textiles'
 'food & beverages' 'management consulting' 'medical practice'
 'accounting' 'performing arts' 'electrical/electronic manufacturing'
 'higher education' 'outsourcing/offshoring'
 'venture capital & private equity' 'writing and editing'
 'mining & metals' 'construction' 'consumer electronics' 'retail'
 'human resources' 'staffing and recruiting' 'farming' 'wholesale'
 'events services' 'import and export'
 'non-profit organization management' 'machinery' 'information services'
 'biotechnology' 'philanthropy' 'law practice' 'real estate'
 'graphic design' 'building materials' 'medical devices' 'consumer go

Create a dictionary called `industry_to_index` where each unique industry is mapped to a unique index starting from 0.

*Hint: Use the `enumerate()` function.*

In [15]:
industry_to_index = {industry: idx for idx, industry in enumerate(industries)}
print(industry_to_index)


{'computer hardware': 0, 'health, wellness and fitness': 1, 'internet': 2, 'financial services': 3, 'mechanical or industrial engineering': 4, 'marketing and advertising': 5, 'hospital & health care': 6, 'research': 7, 'information technology and services': 8, 'computer software': 9, 'oil & energy': 10, 'dairy': 11, 'transportation/trucking/railroad': 12, 'design': 13, 'furniture': 14, 'professional training & coaching': 15, 'hospitality': 16, 'textiles': 17, 'food & beverages': 18, 'management consulting': 19, 'medical practice': 20, 'accounting': 21, 'performing arts': 22, 'electrical/electronic manufacturing': 23, 'higher education': 24, 'outsourcing/offshoring': 25, 'venture capital & private equity': 26, 'writing and editing': 27, 'mining & metals': 28, 'construction': 29, 'consumer electronics': 30, 'retail': 31, 'human resources': 32, 'staffing and recruiting': 33, 'farming': 34, 'wholesale': 35, 'events services': 36, 'import and export': 37, 'non-profit organization management

Create a new column `industry_index` in your DataFrame by mapping the `industry` column to the indices using the `industry_to_index` dictionary.

*Hint: Use the  `map()` function.*

In [16]:
df['industry_index'] = df['industry'].map(industry_to_index)
df.head()


,industry,output,processed_slogan,modified_slogan,industry_index
0,computer hardware,Taking Care of Small Business Technology,taking care of small business technology,computer hardware taking care of small busines...,0
1,"health, wellness and fitness",Build World-Class Recreation Programs,build world class recreation programs,"health, wellness and fitness build world class...",1
2,internet,Most Powerful Lead Generation Software for Mar...,most powerful lead generation software for mar...,internet most powerful lead generation softwar...,2
3,internet,Hire quality freelancers for your job,hire quality freelancers for your job,internet hire quality freelancers for your job,2
4,financial services,"Financial Advisers Norwich, Norfolk",financial advisers norwich norfolk,financial services financial advisers norwich ...,3


Split the DataFrame `df` into training and testing sets, setting aside 20% of the data for the test set. Be sure to set the parameter `stratify=df["industry_index"]`. This ensures that both sets have the same proportion of each class (industry) as in the original dataset, resulting in balanced datasets. Call the training DataFrame `df_train` and the testing DataFrame `df_test`.

In [18]:
class_counts = df["industry_index"].value_counts()
valid_classes = class_counts[class_counts > 1].index
df_filtered = df[df["industry_index"].isin(valid_classes)]

df_train, df_test = train_test_split(
    df_filtered,
    test_size=0.2,
    stratify=df_filtered["industry_index"],
    random_state=42
)

Our classifier will use padded slogan sequences as inputs, similar to input sequences used for the slogan generator. The difference is we will not use sequences that get progressively longer, but instead we will use **complete slogans**. This is because our classifier does not need to learn how to predict what word comes next. It needs the full context of a slogan to learn how to accurately predict the industry.  

The next steps will walk you through how to create these sequences.  

We previously created and fitted a `Tokenizer` object called `tokenizer` while preparing data for the slogan generator. Now, we will reuse it to convert words into numerical indices.  

In the code cell below, use the `texts_to_sequences()` **method** of `tokenizer` to transform the `processed_slogan` column in **both** the `df_train` and `df_test` DataFrames into sequences of numerical indices. Store the results in variables named `X_train` and `X_test`.


In [19]:
X_train = tokenizer.texts_to_sequences(df_train["processed_slogan"])
X_test = tokenizer.texts_to_sequences(df_test["processed_slogan"])

The slogan sequences are of varying lengths. We will need to pad them the same way we did to the input sequences for the slogan generator. The `pad_sequences()` function can ensure the sequences in `slogan_sequences` have the same length.  

In the code cell below, use the `pad_sequences()` function to standardise the `slogan_sequences` lengths. Set the `maxlen` parameter to `max_seq_len`, the `padding` parameter to 0, and assign the resulting padded sequences to the same variables, `X_train` and `X_test`.

In [20]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(X_train, maxlen=max_seq_len, padding='pre')
X_test = pad_sequences(X_test, maxlen=max_seq_len, padding='pre')


We have successfully created training and testing inputs for our model. Now, we will create the outputs - industry categories.

 In the code cell that follows, use `tf.keras.utils.to_categorical()` to apply one-hot encoding to the `industry_index` column of **both** `df_train` and `df_test` DataFrames. Assign the results to a variables named `y_train` and `y_test`.

 *Hint: set the `num_classes` parameter to the total number of industries in the DataFrame. The `industries` variable can be used to find this value.*

In [21]:
num_classes = len(industries)

y_train = to_categorical(df_train['industry_index'], num_classes=num_classes)
y_test = to_categorical(df_test['industry_index'], num_classes=num_classes)


## Slogan Classifier Architecture

Configure the LSTM classifier following these steps:  


1. Create a Sequential model:  
   Use `tf.keras.models.Sequential()` to create a sequential model. This model will consist of an embedding layer, two LSTM layers, and a dense output layer.

2. Add an embedding layer which will convert words into dense vector representations. Configure this layer with:
   > * `total_words` as the vocabulary size.
   > * 100 as the embedding dimension.
   > * `max_seq_len` as the `input_length` (this is the length of the slogans).

3. Add the first LSTM layer. Configure it with:
   > * 150 units.
   > * Set `return_sequences` to `True` to ensure the layer outputs sequences for the next LSTM layer.

4. Add the second LSTM layer which will process the output from the previous LSTM layer. Configure it with:
   > * 100 units.
   > * No need to set `return_sequences` here (it is the final LSTM layer).

5. Add the dense output layer which will classify the data into industries. Configure it with:
   > * The number of unique industries as the number of units.
   > * The `softmax` activation function to get probabilities for each class (industry).

6. Use `Sequential` to arrange all layers in the correct order and complete the architecture of the LSTM model called **class_model**.


In [22]:
class_model = Sequential()

# Embedding layer: vocab size = total_words, embedding dim = 100, input length = max_seq_len
class_model.add(Embedding(input_dim=total_words, output_dim=100, input_length=max_seq_len))

# First LSTM layer with 150 units, returning sequences
class_model.add(LSTM(150, return_sequences=True))

# Second LSTM layer with 100 units
class_model.add(LSTM(100))

# Dense output layer with softmax activation for classification into industries
class_model.add(Dense(len(industries), activation='softmax'))


In the code cell below, compile `class_model` using `categorical_crossentropy` loss, an Adam optimiser, and an appropriate metric of your choice.

In [23]:
class_model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)


## Slogan Classification & Evaluation

In the code cell that follows, fit the compiled model on the inputs and outputs, setting **the number of epochs to 50**.

In [24]:
history = class_model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=64,
    validation_split=0.2
)


Epoch 1/50
54/54 [==============================] - 13s 115ms/step - loss: 4.4952 - accuracy: 0.0670 - val_loss: 4.2499 - val_accuracy: 0.0959
Epoch 2/50
54/54 [==============================] - 5s 97ms/step - loss: 4.2995 - accuracy: 0.0732 - val_loss: 4.2479 - val_accuracy: 0.0959
Epoch 3/50
54/54 [==============================] - 5s 92ms/step - loss: 4.2914 - accuracy: 0.0819 - val_loss: 4.2342 - val_accuracy: 0.0959
Epoch 4/50
54/54 [==============================] - 5s 96ms/step - loss: 4.2866 - accuracy: 0.0819 - val_loss: 4.2299 - val_accuracy: 0.0959
Epoch 5/50
54/54 [==============================] - 5s 87ms/step - loss: 4.2611 - accuracy: 0.0819 - val_loss: 4.1837 - val_accuracy: 0.0959
Epoch 6/50
54/54 [==============================] - 5s 97ms/step - loss: 4.0931 - accuracy: 0.0913 - val_loss: 4.0485 - val_accuracy: 0.1018
Epoch 7/50
54/54 [==============================] - 5s 92ms/step - loss: 3.8631 - accuracy: 0.1141 - val_loss: 4.0495 - val_accuracy: 0.1076
Epoch 8/50


Evaluate the model using the testing set. Add a comment on the model's performance.

In [25]:
test_loss, test_accuracy = class_model.evaluate(X_test, y_test, verbose=2)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# The test accuracy gives you an estimate of how well your model generalizes 
# to unseen data. If the accuracy is high (e.g., above 80%), it indicates the 
# model learned to classify slogans into industries effectively. If it’s 
# lower, you might consider tuning the model, increasing training data, or 
# trying different architectures or preprocessing steps.

34/34 - 0s - loss: 6.7806 - accuracy: 0.1891 - 442ms/epoch - 13ms/step
Test Loss: 6.7806
Test Accuracy: 0.1891


We will now define a function called `classify_slogan` which takes a slogan as input and predicts the industry it belongs to using the trained model, `class_model`.  

Carefully follow the code below and complete the missing parts (indicated by ellipses) as guided by the comments.

In [26]:
def classify_slogan(slogan):
    # Use the preprocess_text to clean the input slogan
    slogan = preprocess_text(slogan)

    # Converting the slogan to a sequence of indices
    sequence = tokenizer.texts_to_sequences([slogan])

    # Pad the sequence using the pad_sequences() function
    padded_sequence = pad_sequences(sequence, maxlen=max_seq_len, padding='pre')

    # Pass padded_sequence into the class_model to get predicted probabilities for each industry
    prediction = class_model.predict(padded_sequence)

    # Use np.argmax() to get the index of the industry with the highest probability
    predicted_index = np.argmax(prediction, axis=1)[0]

    # Return the predicted industry
    return industries[predicted_index]


## Combining the two models

Run the code cell below to combine the two models: we will first generate a slogan for a company in the "internet" industry, then pass the generated slogan to the slogan classifier to see if it correctly classifies it as internet.

In [27]:
industry = "internet"
generated_slogan = generate_slogan(industry)
predicted_industry = classify_slogan(generated_slogan)

print(f"Generated Slogan: {generated_slogan}")
print(f"Predicted Industry: {predicted_industry}")

1/1 [==============================] - 1s 1s/step
Generated Slogan: internet web design and development agency in the uk 's premier lasik and sample blockchain design and services in o communications
Predicted Industry: internet


Compare the results and comment on any differences you notice between the generated slogans and the classifier’s predictions in the markdown cell below.


#Comparison of Generated Slogan and Classifier Prediction

**Generated Slogan:**  
*internet web design and development agency in the uk's premier lasik and sample blockchain design and services in o communications*

**Predicted Industry:**  
*internet*

**Commentary:**  
The classifier correctly identified the industry as "internet," which aligns with the input used for generating the slogan. However, the slogan itself is quite incoherent and includes unrelated terms such as "lasik" (which pertains to eye surgery) and "blockchain" (which, while possibly connected to tech, is not specific to internet services). This suggests that the slogan generator may be pulling in phrases from multiple domains without clear semantic filtering.

The classifier still picked "internet" likely due to the presence of keywords such as "web design," "development," and "communications." This indicates the classifier is more focused on identifying strong domain-specific terms rather than judging the overall coherence of the slogan.

To improve results, the slogan generator could benefit from better industry-specific keyword filtering and more natural language structuring.
